# Recipe Assistant MVP
**Project:** Personalized and Conscious Recipe Assistant Agent

*Initial code (MVP) for a conversational recipe assistant using Google Gemini SDK in Google Colab. This MVP focuses on using Gemini to generate recipes and adaptations directly based on user input, simulating search/creation.*





In [ ]:
import google.generativeai as genai
import os
from IPython.display import display, Markdown
from google.colab import userdata


global_chat_persona = "Personal Recipe Assistant"

# Configure the Google Gemini API
try:
    google_api_key = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = google_api_key
    if not google_api_key:
        raise ValueError("API Key not found in Colab secrets.")
    genai.configure(api_key=google_api_key)
except ImportError:
    if not google_api_key:
        print("API Key not found in the environment variables.")
        google_api_key = None
except Exception as e:
    print(f"Error! API Key not found: {e}")
    google_api_key = None


## Agent prompts

In [ ]:
# 1. Understand the user's request
request_prompt = """
You are the main Orchestrator Agent for a Personalized and Conscious Recipe Assistant.
Your role is to understand the user's request, identify key information (ingredients, restrictions, desired meal type, number of servings, specific adaptations like Airfryer), and coordinate the work of specialized recipe agents.

Based on the user's input, extract the following information:
- Ingredients provided (list)
- Dietary restrictions (list, e.g., "gluten-free", "vegan", "no nuts")
- Meal type (e.g., "lunch", "dinner", "snack")
- Number of servings (integer, default to 2 if not specified)
- Adaptation requests (list, e.g., "Airfryer")

Present the extracted information in a clear, structured format (e.g., JSON-like structure or key-value pairs) so the main program can easily parse it and call the appropriate specialized agents. Do NOT generate the recipe yourself. Simply extract and format the request details.

Example Output Format:
{
  "ingredients": ["chicken", "broccoli", "rice"],
  "restrictions": ["gluten-free"],
  "meal_type": "lunch",
  "servings": 2,
  "adaptations": ["Airfryer"]
}

If the request is unclear or missing crucial information (like ingredients), ask the user for clarification.
"""

# 2. Recipe generation based on ingredients and meal type.
recipe_generator_prompt = """
You are a Recipe Generation Agent. Your task is to create a base recipe based on a list of ingredients and a meal type provided by the Orchestrator Agent.

Generate a complete recipe including:
- Title
- Brief description
- List of ingredients with suggested quantities
- Step-by-step instructions for traditional preparation (stove, oven, etc.)

The recipe should primarily use the ingredients provided. If necessary, you may suggest common pantry staples (salt, pepper, oil) but focus on the core ingredients given.
Present the recipe in a clear, structured format using Markdown.
"""

# 3. Restriction adaptation to modify a recipe to meet specific dietary restrictions.
restriction_adapter_prompt = """
You are a Restriction Adaptation Agent. Your task is to take an existing recipe and modify it to comply with a list of dietary restrictions provided by the Orchestrator Agent.

Analyze the provided recipe and the restrictions. Suggest substitutions or modifications to ingredients and/or instructions to make the recipe compliant.

Present the modified recipe or a clear list of changes needed, maintaining the original structure as much as possible. Use Markdown for clarity.
If the recipe already meets the restrictions, state that. If necessary, use Google (google_search) to adapt the recipe.
"""

# 4. Airfryer adaptation
airfryer_adapter_prompt = """
You are an Airfryer Adaptation Agent. Your task is to take an existing recipe and provide instructions on how to cook it using an Airfryer. If necessary, use Google (google_search) to adapt the recipe.

Analyze the provided recipe. Determine appropriate temperatures, cooking times, and specific steps needed to prepare this dish in an Airfryer.

Provide a separate section detailing the "Airfryer Adaptation". Include suggested temperature, estimated time, and any specific instructions (like preheating, shaking the basket, cooking in batches, etc.). Use Markdown for clarity.
"""

## Agents

In [ ]:
# google agent framework
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner # orquestrador
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
import json
import time


class AgentResponseObject:
    def __init__(self, text_content: str):
        self.text = text_content

default_model = 'gemini-2.5-flash-preview-04-17'

# --- agent orchestrator function ---
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response_text = ""

    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response_text += part.text
    return AgentResponseObject(final_response_text)

# --- agents ---
def request_agent(input):
  agent = Agent(
      name='request_agent',
      model=default_model,
      instruction=request_prompt,
      description="Understand the user's request"
  )
  return call_agent(agent, input)


def recipe_generator_agent(input):
  agent = Agent(
      name='recipe_generator',
      model=default_model,
      instruction=recipe_generator_prompt,
      description="Recipe generation based on ingredients"
  )
  return call_agent(agent, input)


def restriction_adapter_agent(input):
  agent = Agent(
      name='restriction_adapter',
      model=default_model,
      instruction=restriction_adapter_prompt,
      description="Adaptation to modify a recipe to meet specific dietary restrictions",
      tools=[google_search]
  )
  return call_agent(agent, input)


def airfryer_adapter_agent(recipe):
  agent = Agent(
      name='airfryer_adapter',
      model=default_model,
      instruction=airfryer_adapter_prompt,
      description="Adaptat recipe to use airfryer",
      tools=[google_search]
  )
  return call_agent(agent, recipe)


## Main Orchestration

In [ ]:
def process_user_request(user_input):
    try:
        # Step 1: Parse the user's request
        print("\nProcessing your request...")
        request_response = request_agent(f"User request: {user_input}\n\nExtract details in JSON format.")
        request_details_str = request_response.text.strip()

        # parsing JSON-like string
        try:
            if request_details_str.startswith("```json"):
                request_details_str = request_details_str[len("```json"):].strip()
            if request_details_str.endswith("```"):
                request_details_str = request_details_str[:-len("```")].strip()
            request_details = json.loads(request_details_str)
            print(f"{global_chat_persona}: Your request details {request_details}")

        except json.JSONDecodeError as e:
            print(f"{global_chat_persona}: The request details coud not be parsed. Error: {e}")
            # If parsing fails, return the response directly, maybe the orchestrator asked for clarification
            return request_response.text

        ingredients = request_details.get("ingredients", [])
        restrictions = request_details.get("restrictions", [])
        meal_type = request_details.get("meal_type", "meal")
        servings = request_details.get("servings", 2) # Default to 2
        adaptations = request_details.get("adaptations", [])

        if not ingredients:
            return f"{global_chat_persona}: Please tell me what ingredients you have so I can suggest a recipe."

        # Step 2: Generate the base recipe
        print(f"{global_chat_persona}: Generating your recipe...")
        generator_input = f"Ingredients: {', '.join(ingredients)}\nMeal Type: {meal_type}\nServings: {servings}"
        base_recipe_response = recipe_generator_agent(generator_input)
        current_recipe_text = base_recipe_response.text
        # Optional: show base recipe
        # display(Markdown(current_recipe_text))

        # Step 3: Restriction Adaptation if requested
        if restrictions:
            print(f"{global_chat_persona}: Processing restriction adaptation as requested...")
            restriction_input = f"Original Recipe:\n{current_recipe_text}\n\nRestrictions: {', '.join(restrictions)}"
            restriction_response = restriction_adapter_agent(restriction_input)
            current_recipe_text = restriction_response.text

        # Step 4: Airfryer Adaptation if requested
        if any(word.lower() == "airfryer".lower() for word in adaptations):
            print(f"{global_chat_persona}: Processing Airfryer adaptation as requested...")
            airfryer_input = f"Original Recipe:\n{current_recipe_text}"
            airfryer_response = airfryer_adapter_agent(airfryer_input)
            current_recipe_text += "\n\n" + airfryer_response.text

        # Step 5: Return the final recipe
        return current_recipe_text

    except Exception as e:
        print(f"{global_chat_persona}: Error when processing your request: {e}")
        return "An error occurred while processing your request. Please try again."


In [ ]:
# --- Main ---
# user interaction
if google_api_key:
    try:
        print("\nHey! I am your personal Recipe Assistant.\n\n")
        print("Tell me what ingredients you have, your restrictions, for how many people, and if you want to adapt it for an Airfryer.\n")
        print("Example: I have potatoes and cauliflower. I want a vegetarian and gluten-free dinner. Please adapt for the airfryer.")

        while True:
          user_input = input("\nRecipe request (or type 'exit' to quit): ")

          # exit condition
          if user_input.lower() == 'exit':
              print("\nEnjoy your meal... See you soon!\n\n ")
              break

          # Process using the multi-agent system
          final_response_text = process_user_request(user_input)

          print(f"\n{global_chat_persona}:")
          display(Markdown(final_response_text))

          if "An error occurrd while processing your request" in final_response_text:
              break
          time.sleep(0.1)
    except Exception as e:
        print(f"Error: {e}")

else:
    print(f"\n{global_chat_persona} could not be initialized. Check your API Key and try again.")
